# Objectifs
Implémenter la rétroprojection filtrée 2D en géométrie parallèle. L'algorithme consiste à rétroprojeter:
\begin{equation}
f(\underline{x})=\int_0^\pi p^*_\phi(\underline{x}\cdot\underline{\alpha}_\phi)\mathrm{d}\phi
\end{equation}
les projections filtrées avec un filtre rampe:
\begin{equation}
p^*_\phi(s) = \int_{-\infty}^\infty \lvert \sigma \rvert \hat p_\phi(\sigma)e^{2\pi i\sigma s}\mathrm{d}\sigma
\end{equation}
avec $\underline\alpha_\phi=(\cos \phi,\sin\phi)$ l'orientation du détecteur et $\hat p_\phi$ la transformée de Fourier 1D de $p_\phi$.

Le dessin ci-dessous ilustre les notations utilisées dans ce notebook:

![parallel_fig.png](parallel_fig.png)

Dans un deuxième temps nous nous intéresserons à un algorithme de reconstruction itératif: Maximum-Likelihood Expectation-Maximization (MLEM). 

Enfin, il s’agira de comparer les temps de calcul et la qualité des différentes reconstructions de chacune des deux méthodes.

# Données
Nous allons travailler sur une coupe du fantôme de Shepp Logan 3D.
Le code utilisé pour produire l'image de `réference` est donné ci-dessous:

In [ ]:
import numpy as np
import itk
from itk import RTK as rtk

# Define the image type, we will use 3D float images in these hands-on sessions
ImageType = itk.Image[itk.F,3]

# Create an image filled with 0 in which we will draw the phantom
srcdraw = rtk.ConstantImageSource[ImageType].New()
srcdraw.SetConstant(0.) # Note that this is useless because 0 is the default
srcdraw.SetSize([256,1,256])
srcdraw.SetSpacing([1,1,1])
srcdraw.SetOrigin([-127.5,0.,-127.5])

# Draw the Shepp Logan in it
sldraw = rtk.DrawSheppLoganFilter[ImageType, ImageType].New()
sldraw.SetInput(srcdraw.GetOutput())
sldraw.SetPhantomScale(128)
sldraw.SetOriginOffset([0.,0.25,0.])
sldraw.InPlaceOff()
sldraw.Update()
itk.imwrite(sldraw.GetOutput(),'ref.mha')
reference = itk.GetArrayFromImage(sldraw.GetOutput()).squeeze()

%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
x1=np.linspace(srcdraw.GetOrigin()[0],
               srcdraw.GetOrigin()[0]+(srcdraw.GetSize()[0]-1)*srcdraw.GetSpacing()[0],
               num=reference.shape[1])
x2=np.linspace(srcdraw.GetOrigin()[2],
               srcdraw.GetOrigin()[2]+(srcdraw.GetSize()[2]-1)*srcdraw.GetSpacing()[2],
               num=reference.shape[0])
plt.imshow(reference, clim=[1,1.05], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.xlabel('x1 (mm)')
plt.ylabel('x2 (mm)')
plt.title('Reference')
plt.colorbar()
plt.show();

et la fonction permettant de calculer les projections est la suivante:

In [ ]:
def RadonTransformSheppLogan(phideg):
    geometry = rtk.ThreeDCircularProjectionGeometry.New()
    for angle in phideg:
        # The convention in the courses is opposite of that of RTK for the rotation angle.
        # We simply use the courses convention and convert the angle to RTK's convention
        geometry.AddProjection(1000,0,-angle)

    # Create an image filled with 0 in which we will draw the phantom
    srcproj = rtk.ConstantImageSource[ImageType].New()
    srcproj.SetSize([reference.shape[1],1,len(phideg)])
    srcproj.SetSpacing(srcdraw.GetSpacing())
    srcproj.SetOrigin([srcdraw.GetOrigin()[0],0.,0.]) 

    # Project the Shepp Logan in it
    slproj = rtk.SheppLoganPhantomFilter[ImageType, ImageType].New()
    slproj.SetInput(srcproj.GetOutput())
    slproj.SetGeometry(geometry)
    slproj.SetPhantomScale(128)
    slproj.SetOriginOffset([0.,0.25,0.])
    slproj.Update()
    sinogram = itk.GetArrayFromImage(slproj.GetOutput()).squeeze()
    s=np.linspace(srcproj.GetOrigin()[0],
              srcproj.GetOrigin()[0]+(srcproj.GetSize()[0]-1)*srcproj.GetSpacing()[0],
              num=srcproj.GetSize()[0])
    return sinogram, s

**Exercice:** Tracer le sinogramme en précisant $N_\phi$, le nombre d'angles choisi dans l'intervalle $[0°,180°[$

In [ ]:
#Nombre d'angles à prendre en comtpe
Nphi = 30
#Création des différents angles
phis = np.linspace(0,180, Nphi)
#Création du sinnogramme
sinogram, s = RadonTransformSheppLogan(phis)
#Affichage du sinnogramme
plt.imshow(np.transpose(sinogram))
plt.xlabel('angle $\phi$ en degré')
plt.ylabel('u')
plt.title("Sinnogramme du Fantome de Shepp Logan")

# Théorème coupe-projection

**Exercice** Vérifier le théorème coupe-projection pour la projection $p_0$:
\begin{equation}
\hat{p}_0(\sigma)=\hat{f}(0,\sigma)
\end{equation}
Vous pouvez utiliser [numpy.fft.rfft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.rfft.html) pour la transformée de Fourier 1D, [numpy.fft.rfft2](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.rfft2.html) pour la transformée de Fourier 2D et [numpy.absolute](https://docs.scipy.org/doc/numpy/reference/generated/numpy.absolute.html) pour calculer le module d'un nombre complexe.

In [ ]:
p0 = sinogram[0]        #Projection du fantôme de Shepp Logan suivant l'angle phi = 0
P0 = np.fft.rfft(p0)    #Transformee de Fourier de la projection

REFERENCE = np.fft.rfft2(reference)     #Transformée de Fourier 2D du fantome de Shepp Logan



#Visualisation
plt.figure(1, figsize=[20.0,10.0])
plt.subplot(221)
plt.plot(p0)
plt.title("Projection du fantome de Shepp Logan suivant l'angle $\phi$ = 0°")

plt.subplot(222)
plt.plot(np.absolute(P0))
plt.title("Transformée de Fourier de la projection suivant l'angle $\phi$ = 0°")

plt.subplot(223)
plt.imshow(np.absolute(REFERENCE))
plt.title("Transformée de Fourier 2D du fantome de Shepp Logan")

plt.subplot(224)
plt.plot(np.absolute(REFERENCE)[0])
plt.title("Transformée de Fourier 2D évaluée en 0")

plt.figure(2)
plt.plot(np.absolute(P0) - np.absolute(REFERENCE)[0])
plt.title("Différence des deux transformées de Fourier")


# Rétroprojection
On cherche maintenant à inverser la projection, i.e. à passer du domaine de Radon au domaine image.

**Exercice:** Implémenter dans un premier temps la rétroprojection simple de la transformée de Radon:
\begin{equation}
f(\underline{x})=\int_0^\pi p_\phi(\underline{x}\cdot\underline{\alpha}_\phi)\mathrm{d}\phi = \int_0^\pi p_\phi(x_1 cos(\phi)+x_2 sin(\phi))\mathrm{d}\phi
\end{equation}
qui après discrétisation donne:
\begin{equation}
f(x_1, x_2)=\Delta \phi \sum_{k=0}^{N_\phi-1} p_{\phi_k}(x_1 cos(\phi_k) + x_2 sin(\phi_k))
\end{equation}

Pour l'algorithme de la rétroprojection filtrée, la meilleure solution pour implémenter l'opérateur de rétroprojection est d'utiliser un *pixel-based* rétroprojecteur:
- Calculer les coordonnées 2D de chaque pixel de la fonction discretisée $f$. 
- Regarder où se projette chaque pixel sur chaque projection 1D $p_\phi$.
- Faire une interpolation linéraire des projections à ces posisitions.
- Ajouter la valeur interpolée au pixel.

![pixel-based](pixel_based_backprojection.png)

Vous pouvez utiliser la fonction [numpy.meshgrid](https://docs.scipy.org/doc/numpy/reference/generated/numpy.meshgrid.html) pour obtenir une grille de coordonnées à partir des coordonnées des vecteurs $x_1$ et $x_2$ (voir leur définition dans la section Donnée). Utilisez l'option`indexing='xy'` pour être cohérent avec la `réference`. Vous pouvez utiliser [numpy.interp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html) pour l'interpolation linéaire.

In [ ]:
from math import cos, sin, pi


def BackprojectionSimple(Nphi,sinogram,s,x1,x2):


    #Initialisation de la reconstruction
    rec = np.zeros(np.shape(reference))
    #Céations des grilles X1 et X2 des coordonnées pixelliques
    X1,X2 = np.meshgrid(x1,x2,indexing='xy')

    inc = pi/Nphi
    for nb in range(Nphi) :

        phi = inc * nb

        proj = sinogram[nb,:]

        pCoords = X1*cos(phi) + X2*sin(phi)

        pValues = np.interp(pCoords, x1, proj)

        rec += pValues        

    return inc*rec


f_result = BackprojectionSimple(Nphi,sinogram,s,x1,x2)

#Visualisation
plt.imshow(f_result) # ,clim=[1,1.05], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.colorbar()

Le code suivant réalise la rétroprojection simple en utilisant RTK.

In [ ]:
def BackprojectionRTK(phideg):
    geometry = rtk.ThreeDCircularProjectionGeometry.New()
    for angle in phideg:
        # The convention in the courses is opposite of that of RTK for the rotation angle.
        # We simply use the courses convention and convert the angle to RTK's convention
        geometry.AddProjection(1000,0,-angle)
        
    # Recreate projections with 3 slices to accomodate bilinear interpolation
    srcproj = rtk.ConstantImageSource[ImageType].New()
    srcproj.SetSize([reference.shape[1],3, len(phideg)])
    srcproj.SetOrigin([srcdraw.GetOrigin()[0],-1.*srcdraw.GetSpacing()[1],0])
    srcproj.SetSpacing(srcdraw.GetSpacing())
    slproj = rtk.SheppLoganPhantomFilter[ImageType, ImageType].New()
    slproj.SetInput(srcproj.GetOutput())
    slproj.SetGeometry(geometry)
    slproj.SetPhantomScale(128)
    slproj.SetOriginOffset([0.,0.25,0.])
    slproj.Update()

    # Backproject
    rtkbp = rtk.BackProjectionImageFilter[ImageType, ImageType].New()
    rtkbp.SetInput(0,srcdraw.GetOutput())
    rtkbp.SetInput(1,slproj.GetOutput())
    rtkbp.SetGeometry(geometry)
    rtkbp.Update()
    rtkbp = itk.GetArrayFromImage(rtkbp).squeeze()
    rtkbp *= np.abs(phideg[1]-phideg[0])
    
    return rtkbp

**Exercice:** Valider votre rétroprojection en la comparant à celle de RTK. La méthode de rétroprojection simple est-elle satisfaisante ? Expliquer pourquoi.

In [ ]:
rtkRes = BackprojectionRTK(phis)

plt.figure(1)
plt.imshow(rtkRes)
plt.colorbar()


# Filtre Rampe
La version analytique du filtre rampe $\hat h(\sigma)=|\sigma|$ n'est pas utilisable dans notre cas du fait que les signaux numériques possèdent un spectre limité. Le filtre s'exprime donc par: 
\begin{equation}
\hat h_\mathrm{BL}(\sigma)=\left\{
\begin{array}{ll}%
\lvert \sigma \rvert &\mathrm{si}\ \lvert \sigma \rvert <\Sigma\\
0 & \mathrm{sinon,}\\
\end{array}
\right.
\end{equation}
où $\Sigma=1/2\Delta_s$ est la frèquence maximum respectant la condition de Nyquist-Shannon pour un signal échantillonée à $\Delta_s$. Il existe plusieurs implémentations du filtre rampe verifiant cette condition, une des plus communes est: 
\begin{equation}
h_\mathrm{BL}(s)=\dfrac{1}{2\Delta_s^2}\dfrac{\sin 2\pi s/2\Delta_s}{2\pi s/2\Delta_s}-\dfrac{1}{4\Delta_s^2}\left(\dfrac{\sin\pi s/2\Delta_s}{\pi s/2\Delta_s}\right)^2
\end{equation}
qui après discrétisation donne:
\begin{equation}
h_\mathrm{BL}(n\Delta_s)=
\begin{cases}
\dfrac{1}{4\Delta_s^2} &\mathrm{si}\ n=0,\\
0 &\mathrm{si}\ n\ \mathrm{est\ pair},\\
-\dfrac{1}{\left(n\pi \Delta_s\right)^2} &\mathrm{si}\ n\ \mathrm{est\ impair}.\\
\end{cases}
\end{equation}

Le code suivant retourne ce filtre à la fois dans le domaine spatial et fréquentiel:

In [ ]:
def RampFilterBL(s, zeropad=2):
    rampbl = np.zeros(s.size*zeropad)
    ds=s[1]-s[0]
    rampbl[s.size*zeropad//2-1]=1/(4*ds**2)
    idxodd = np.concatenate((np.flip(-1*np.arange(1,rampbl.size//2,2)),np.arange(1,rampbl.size//2,2)))
    rampbl[0::2]=-1/(idxodd*np.pi*ds)**2
    rampbl*=ds
    rampbl_fft = np.absolute(np.fft.rfft(rampbl))
    
    return rampbl, rampbl_fft

**Exercice:** En utlisant la fonction précédente, implémentez la rétroprojection filtrée. Vous pouvez utiliser la fonction [numpy.fft.irfft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.irfft.html) pour la transformée de Fourier 1D inverse, et [numpy.fft.rfftfreq](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.rfftfreq.html) pour récupérer l'échantillonage des fréquences dans le domaine de Fourier.
Comparer l'image reconstruite avec la référence (vous pourrez notamment afficher un profil). Expliquer l'intérêt et l'effet du filtre rampe lors de la rétroprojection.

In [ ]:
# freqs = np.fft.rfftfreq(len(s)-1, s[1] - s[0])
def BackprojectionFiltered(Nphi, sinogram, s, x1, x2) :
    r,R = RampFilterBL(s)

    PROJ = np.fft.rfft(sinogram,axis=1, n= sinogram.shape[1]*2)

    MULT = np.multiply(R,PROJ)

    mult = np.fft.irfft(MULT)[:,:sinogram.shape[1]]

    rec = BackprojectionSimple(Nphi,mult,s,x1,x2)

    return rec


sref = np.shape(reference)
rec = BackprojectionFiltered(Nphi, sinogram, s, x1, x2)

plt.figure(figsize=[10.0,7.0])
plt.subplot(221)
plt.imshow(rec,clim=[1.0,1.05], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.plot([x2[0],x2[-1]], [x1[int(sref[0]/2)], x1[int(sref[0]/2)]], '-r')
plt.title("Reconstruction de la reference")

plt.subplot(222)
plt.imshow(reference, clim=[1.0, 1.05], cmap = plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.plot([x2[0],x2[-1]], [x1[int(sref[0]/2)], x1[int(sref[0]/2)]], '-r')
plt.title("Reference")

plt.subplot(212)
plt.plot(rec[int(sref[0]/2),:], 'r')
plt.plot(reference[int(sref[0]/2),:], 'b')
# plt.plot(rec[int(sref[0]/2),:] - reference[int(sref[0]/2),:], 'g')
plt.axis([0, sref[1], 1, 1.05])
plt.title("Visualisation de deux profils au centre de l'image (zoom pertinent sur les donnees)")



**Exercice:** Comparer les qualités d'images reconstruites, les PSNR obtenus et le temps de reconstruction en fonction de $N_\phi$. Pour mesurer le temps de calcul vous pouvez utiliser la fonction [time.time()](https://docs.python.org/3/library/time.html).

Rappel: $\mathrm{PSNR}= 10 \log_{10}(\frac{\mathrm{MAX}^2_I}{\mathrm{MSE}})$

N'oubliez pas de normalier vos images avant de quantifier leur différence.

In [ ]:
import time
tps = []
psnr = []
N = range(10,500,10)
for n in N:
    #Création des différents angles
    phisVal = np.linspace(0,180, n)
    t1 = time.time()
    #Création du sinnogramme
    sinog, sVals = RadonTransformSheppLogan(phisVal)
    #Retroprojection
    newRec = BackprojectionFiltered(n, sinog, s, x1, x2)
    
    #Calcul du temps
    t2 = time.time()
    tps.append(t2-t1)
    #Calcul du psnr
    MSE = np.mean((rec/np.max(np.max(newRec)) - reference/np.max(np.max(reference)))**2)
    PSNR = 10*np.log10(1/MSE)
    psnr.append(PSNR)


plt.figure()
plt.subplot(211)
plt.plot(N, tps)
plt.title("Evolution du temps de calcul en fonction du nombre de projections")
plt.subplot(212)
plt.plot(N, psnr)
plt.title("Evolution du PSNR en fonction du nombre de projections")
plt.show()


**Exercice:** Rajouter un bruit additif Gaussien de moyenne nulle au sinogramme de départ en utilisant la fonction [np.random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html). Pour un $N_\phi$ fixé, tracez l’évolution du PSNR en fonction de la variance du bruit.

In [ ]:
newN = 100
#Création des différents angles
phisVal = np.linspace(0,180, newN)
t1 = time.time()
#Création du sinnogramme
sinog, sVals = RadonTransformSheppLogan(phisVal)

sSinog = sinog.shape

psnrNoise = []
Sig = np.linspace(0.5, 50, 500)
for sigma in Sig:
    randomVals = np.random.normal(0, sigma, int(sSinog[0]*sSinog[1]))
    randomVals = np.reshape(randomVals, sSinog)

    noiseSinog = sinog + randomVals
    #Retroprojection
    noiseRec = BackprojectionFiltered(newN, noiseSinog, sVals, x1, x2)
    
    #Calcul du psnr
    MSE = np.mean((noiseRec/np.max(np.max(noiseRec)) - reference/np.max(np.max(reference)))**2)
    PSNR = 10*np.log10(1/MSE)
    psnrNoise.append(PSNR)

plt.plot(Sig, psnrNoise)
plt.title("Evolution du PSNR en fonction de la valeur de $\sigma$")
plt.show()


# Bonus (si vous avez le temps)
Nous avons vu dans le cours que le filtre rampe favorise l'amplification des hautes fréquences et peut introduire du bruit dans l'image reconstruite. Une solution est d'appliquer un filtre passe-bas appelé fenêtre d'apodisation pour atténuer les hautes-fréquences. Le filtre de Hann est couramment utilisé à cet effet:
\begin{equation}
\hat w(\sigma)=\left\{
\begin{array}{ll}%
0.5*(1+\cos(\frac{\pi \sigma}{\sigma_c})) &\mathrm{si}\ \sigma <\sigma_c\\
0 & \mathrm{sinon}\\
\end{array}
\right.
\end{equation}

**Exercice:** Ajouter le filtre de Hann à votre rétroprojection filtrée et comparer les résultats obtenue pour différentes fréquences de coupure.

# MLEM
Nous supposons que l'émission des photons suit une distribution de Poisson, alors, si $x$ représente le vecteur image $x = (x_i,..,x_n)^t$ et $y = (y_1,..,y_m)^t$ les projections mesurées, la fonction de vraisemblance, qui est la probabilité d'obtenir $y$ si l'image est $x$, est:
		\begin{equation}
			P_L(y/x) = \prod_{i=1}^m \frac{\sum_{j=1}^n (a_{ij}x_j)^{y_i}}{y_i!} \exp(-\sum_{j=1}^n a_{ij}x_j)
		\end{equation}
où $a_{ij}$ représente la probabilité qu'un photon émis par le voxel j soit détecté par le pixel i et $\sum_{i=1}^m a_{ij} = 1$. 

Afin d'estimer x à partir de y, une approche possible est de maximiser $P_L(y/x)$ ou de façon équivalente:
		\begin{equation}
			\max_{x\ge0}L(x)=\sum_{i=1}^m (y_i \ln(\sum_{j=1}^n a_{ij}x_j) - \sum_{j=1}^n a_{ij}x_j)
		\end{equation} 

L'algorithme MLEM cherche une solution qui maximise cette fonction de vraisemblance entre les projections mesurées et les projections de l'image. L'équation de mise à jour est:
		\begin{equation}
			x^{next}_{j} = \frac{x^{current}_{j}}{{\sum_{i} a_{ij}}} {\sum_{i} a_{ij}} \frac{{y_{i}}}{{{\sum_{j} a_{ij}}}x^{current}_{j}}
		\end{equation}
où $\sum_{i} a_{ij}$ représente l'opérateur de rétroprojection et $\sum_{j} a_{ij}$ l'opérateur de projection.

Une implémentation d'un opérateur de projection est donnée par la fonction suivante:

In [ ]:
def ForwardProjectionRTK(volume, phideg):
    geometry = rtk.ThreeDCircularProjectionGeometry.New()
    for angle in phideg:
        # The convention in the courses is opposite of that of RTK for the rotation angle.
        # We simply use the courses convention and convert the angle to RTK's convention
        geometry.AddProjection(1000,0,-angle)
    
    # Create an image filled with 0 in which we write the projections
    srcproj = rtk.ConstantImageSource[ImageType].New()
    srcproj.SetSize([reference.shape[1],1,len(phideg)])
    srcproj.SetSpacing(srcdraw.GetSpacing())
    srcproj.SetOrigin([srcdraw.GetOrigin()[0],0,0.])
    srcproj.Update()
    
    volume = volume[:, np.newaxis, :]
    volume = np.append(volume, np.zeros((volume.shape[0],2,volume.shape[2])), axis=1)
    itkVolume = itk.GetImageFromArray(volume.astype(np.float32))
    itkVolume.SetSpacing([1,1,1])
    itkVolume.SetOrigin([-127.5,0.,-127.5])

    jfp = rtk.JosephForwardProjectionImageFilter[ImageType, ImageType].New()
    jfp.SetInput(0, srcproj.GetOutput())
    jfp.SetInput(1, itkVolume)
    jfp.SetGeometry(geometry)
    jfp.Update()
    jfp = itk.GetArrayFromImage(jfp.GetOutput()).squeeze()
    
    return jfp  

**Exercice:** En utilisant le projecteur implémenté ci-dessus et votre rétroprojecteur, créer une fonction réalisant la reconstruction par la méthode MLEM.

Dans un premier temps, le nombre d'itération sera fixé.

In [ ]:
def MLEM(f_estime, Nphi, sinogram, s, x1, x2):
    phis = np.linspace(0,180, Nphi)
    proj_n = np.zeros(sinogram.shape)
    proj_Erreur = np.zeros(sinogram.shape)
    f_erreur = np.zeros(f_estime.shape)

    proj_n = ForwardProjectionRTK(f_estime, phis) # projection pn de fn

    proj_Erreur = np.minimum(sinogram / (proj_n+0.0001), 10)

    # print("Proj", np.min(proj_n),np.shape(proj_n))
    # print("PErreur", np.min(proj_Erreur), np.max(proj_Erreur), np.shape(proj_Erreur))

    # plt.figure()
    # plt.subplot(141)
    # plt.imshow(proj_n)
    # plt.colorbar()
    # plt.subplot(142)
    # plt.imshow(sinogram)
    # plt.colorbar()
    # plt.subplot(143)
    # plt.imshow(proj_Erreur, clim=[0,1])
    # plt.colorbar()

    f_erreur = BackprojectionSimple(Nphi, proj_Erreur, s, x1, x2)
    # print("F_err", np.min(f_erreur), np.max(f_erreur), np.shape(f_erreur), np.shape(f_estime))

    f_estime = np.multiply(f_estime,f_erreur)/pi

    return f_estime

In [ ]:

#Initialisation de la reconstruction
f_estime = np.ones(np.shape(reference))
#Nombre d'angles à prendre en comtpe
Nphi = 1000
#Création des différents angles
phis = np.linspace(0,180, Nphi)

#Création du sinnogramme
sinogram, s = RadonTransformSheppLogan(phis)


"""
print(np.shape(f_estime))
print(f_estime)

f_erreur = np.zeros(np.shape(reference))
proj_n = ForwardProjectionRTK(reference, phis)
print(np.shape(proj_n))
inc = pi/Nphi

for nb in range(2) :

    phi = inc * nb

    pCoords = X1*cos(phi) + X2*sin(phi)
    print(np.where(proj_n[nb,:] == 0)[0])
    proj_test = proj_n[nb,:]

    pValues = np.interp(pCoords, x1, proj_test)
    print(np.max(pValues))

    f_erreur += pValues
print(f_erreur)

#Visualisation
plt.imshow(f_erreur) #, clim=[600,650], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.colorbar()
"""
plt.figure(figsize=[20, 10])
for loop in range(40):

    f_estime = MLEM(f_estime, Nphi, sinogram, s, x1, x2)

    if (loop+1)%5 == 0:
        
        plt.subplot(2,4,int((loop+1)//5))
        plt.imshow(f_estime, clim = [1.0, 1.05],cmap = plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
        plt.title("Resultats apres " + str(loop+1) + " iterations")
        plt.colorbar()

"""
    #print(np.shape(proj_n))
    #print(proj_n)
    #Initialisation de la reconstruction
    f_erreur = np.zeros(np.shape(reference))
    #Céations des grilles X1 et X2 des coordonnées pixelliques
    X1,X2 = np.meshgrid(x1,x2)

    inc = pi/Nphi
    # print(loop)

    for nb in range(Nphi) :

        phi = inc * nb

        #print(np.where(proj_n[nb,:] == 0)[0])
        id_non_nul = np.nonzero(proj_n[nb,:])
        proj_erreur = np.zeros(np.shape(sinogram[nb,:]))
        proj_erreur[id_non_nul] = np.divide(sinogram[nb,id_non_nul],proj_n[nb,id_non_nul]) 

        pCoords = X1*cos(phi) + X2*sin(phi)

        pValues = np.interp(pCoords, x1, proj_erreur)

        f_erreur += pValues
    
    #f_estime = np.dot( f_estime , inc * f_erreur )
    f_estime =  inc * f_estime * f_erreur 
    #print(f_estime)






    p = ForwardProjectionRTK(f_estime, phis) # projection pn de fn
    p_log = np.log(p)
    diff = p_log - p
    
    #Initialisation de la reconstruction
    log_vrai = np.zeros(np.shape(reference))
    #Céations des grilles X1 et X2 des coordonnées pixelliques
    X1,X2 = np.meshgrid(x1,x2)

    inc = pi/Nphi

    for nb in range(Nphi) :

        phi = inc * nb

        id_non_nul = np.nonzero(proj_n[nb,:])
        proj = np.zeros(np.shape(sinogram[nb,:]))
        proj[id_non_nul] = np.multiply(sinogram[nb,id_non_nul],diff[nb,id_non_nul]) - diff[nb,id_non_nul]

        pCoords = X1*cos(phi) + X2*sin(phi)

        pValues = np.interp(pCoords, x1, proj)

        log_vrai += pValues
    # print(np.mean(np.mean(np.abs(log_vrai))))


    erreur = (sinogram-p)**2
    # print(np.mean(np.mean(erreur)))



# plt.figure(loop+1)
# #Visualisation
# plt.imshow(f_estime, clim=[1,1.05], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')#,clim=[1.0,1.05], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower') #, clim=[600,650], cmap=plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.colorbar()

# print(f_estime)

# sommei rij fcurrent
"""






**Exercice:** Vérifier que l'algorithme MLEM maximise bien la fonction de coût en traçant sont évolution en fonction du nombre d'itérations. Tracer aussi l'évolution de l'érreur quadratique moyenne pour vérifier que le volume reconstruit se rapproche bien de la référence.

En déduire un critère de convergence de l'algorithme plus pertinent qu'un nombre fixé d'itérations.

In [ ]:

def getLossValue(sinogram, sinogEstim):
    logSE = np.log(sinogEstim + 0.00001)
    L = np.sum(np.sum(np.multiply(sinogram, logSE) - sinogEstim))

    return L

def getMSE(sinogram, sinogEstim):
    M = np.mean(np.square(sinogram/np.max(np.max(sinogram))-sinogEstim/np.max(np.max(sinogEstim))))
    return M

#Initialisation de la reconstruction
f_estime = np.ones(np.shape(reference))
#Nombre d'angles à prendre en comtpe
Nphi = 1000
#Création des différents angles
phis = np.linspace(0,180, Nphi)
#Création du sinnogramme
sinogram, s = RadonTransformSheppLogan(phis)

L = []
MSEVals = []
MSEValsIm = []
nbIter = 100
for loop in range(nbIter):
    f_estime = MLEM(f_estime, Nphi, sinogram, s, x1, x2)
    sinogEstim = ForwardProjectionRTK(f_estime, phis)
    L.append(getLossValue(sinogram, sinogEstim))
    MSEVals.append(getMSE(sinogram, sinogEstim))
    MSEValsIm.append(getMSE(reference, f_estime))


print("Ecart type de la MSE entre 10 et " + str(nbIter) + " iterations : " + str(np.std(MSEVals[10:])) + "\nEcart type de la LogVraisemblance entre 10 et " + str(nbIter) + " iterations : " + str(np.std(L[10:])))

plt.figure(figsize=[15, 10])
plt.subplot(211)
plt.plot(np.linspace(1,nbIter, nbIter), L)
plt.plot(np.argwhere(L==np.max(np.max(L))), np.max(np.max(L)), 'xr')
plt.title("Evolution de la fonction de cout")

plt.subplot(223)
plt.plot(np.linspace(1,nbIter, nbIter), MSEVals)
ind = np.argwhere(MSEVals <= 0.0001+np.min(np.min(MSEVals)))[0]
plt.plot(ind, MSEVals[int(ind)], 'xr')
plt.title("Evolution de l'erreur quadratique moyenne des sinnogrammes")

plt.subplot(224)
plt.plot(np.linspace(1,nbIter, nbIter), MSEValsIm)
plt.plot(np.argwhere(MSEValsIm==np.min(np.min(MSEValsIm))), np.min(np.min(MSEValsIm)), 'xr')
plt.title("Evolution de l'erreur quadratique moyenne des volumes")
plt.show()



# Bilan
**Exercice:** En fixant les mêmes paramètres (bruit, taille de l’image, nombre de projections) comparer les reconstructions obtenues (PSNR, temps de calcul, perception visuelle) par les deux méthodes implémentées.

In [ ]:
NphiFix = 50
sigmaNoise = 0.05

#Création des différents angles
phisFix = np.linspace(0,180, NphiFix)
#Création du sinnogramme
sinogramFix, sFix = RadonTransformSheppLogan(phisFix)
sRef = reference.shape

#Bruitage des données
randomValsBil = np.random.normal(0, sigmaNoise, int(sinogramFix.shape[0]*sinogramFix.shape[1]))
# print(np.mean(randomValsBil), np.std(randomValsBil))
randomValsBil = np.reshape(randomValsBil, sinogramFix.shape)
noiseSinogBil = np.maximum(sinogramFix + randomValsBil, 0 )

# print(np.min(randomValsBil), np.max(randomValsBil))

#Initialisation des caracteristiques
T, Psnr = [], []

#Calcul de la retroprojection filtree
t1 = time.time()
noiseRec = BackprojectionFiltered(NphiFix, np.copy(noiseSinogBil), sFix, x1, x2)
t2 = time.time()
T.append(t2-t1)
mseFilt = getMSE(noiseRec, reference)
Psnr.append(10*np.log10(mseFilt))

#Calcul de la retroprojection avec MLEM
t1bis = time.time()
f_estime_bil = np.ones(np.shape(reference))
epsilon = 1e-5

prev = 1000
i = 0
P = []
while (prev - getMSE(np.copy(noiseSinogBil), ForwardProjectionRTK(np.copy(f_estime_bil), phisFix)) > epsilon) and (i < 200) :
    prev = getMSE(np.copy(noiseSinogBil), ForwardProjectionRTK(np.copy(f_estime_bil), phisFix))
    f_estime_bil = MLEM(np.copy(f_estime_bil), NphiFix, np.copy(noiseSinogBil), sFix, x1, x2)
    P.append(prev)
    i+=1
print(i)

# plt.figure()
# plt.plot(P)
t2bis = time.time()
T.append(t2bis - t1bis)
mseMLEM = getMSE(f_estime_bil, reference)
Psnr.append(10*np.log10(mseMLEM))

plt.figure(figsize  = [20, 10])
plt.subplot(231)
plt.imshow(reference, clim = [1.0, 1.05],cmap = plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.plot([x2[0],x2[-1]], [x1[int(sRef[0]/2)], x1[int(sRef[0]/2)]], '-r')
plt.title("Image de reference")

plt.subplot(232)
plt.imshow(noiseRec, clim = [1.0, 1.05],cmap = plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.text(x1[5], x2[-25], "Time to run = " + str(round(T[0],2)) + "\nPSNR = " + str(round(Psnr[0],2)), fontdict={'color':'white'})
plt.plot([x2[0],x2[-1]], [x1[int(sRef[0]/2)], x1[int(sRef[0]/2)]], '-r')
plt.title("Image reconstruite avec retroprojection filtree")

plt.subplot(233)
plt.imshow(f_estime_bil, clim = [1.0, 1.05],cmap = plt.cm.gray, extent=(x1[0],x1[-1],x2[0],x2[-1]), origin='lower')
plt.text(x1[5], x2[-25], "Time to run = " + str(round(T[1],2)) + "\nPSNR = " + str(round(Psnr[1],2)), fontdict={'color':'white'})
plt.plot([x2[0],x2[-1]], [x1[int(sRef[0]/2)], x1[int(sRef[0]/2)]], '-r')
plt.title("Image reconstruite avec MLEM")

plt.subplot(212)
plt.plot(reference[int(sRef[0]/2),:], 'r')
plt.plot(noiseRec[int(sRef[0]/2),:], 'b')
plt.plot(f_estime_bil[int(sRef[0]/2),:], 'g')
plt.axis([0, sRef[1], 1, 1.05])
plt.title("Visualisation des profils au centre de l'image (zoom pertinent sur les donnees)")
plt.legend(["Image de reference", "Retroprojection filtree", "MLEM"])

